In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch
import numpy as np
import time
import os
from utils import HDDataset
from model import BModel
from torchvision import datasets, transforms
import scipy
from encoder_1d import RandomFourierEncoder
import main2_1d as main2 
import importlib
importlib.reload(main2)
import csv
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset, TensorDataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
gorder = 2
dim = 15000
gamma = 0.3
lr = 0.01
classes = 10
channels = 3
epochs = 3
train = main2.train
test = main2.test

In [4]:
def load_ucr(file):
    data = np.loadtxt(file)
    X = data[:, 1:]
    y = data[:, 0]
    y = np.where(y == 1, 1, 0)  # convert labels to 0/1
    return X, y

# Adjust file paths to your local files
X_train, y_train = load_ucr("../FordA_TRAIN.txt")
X_test, y_test = load_ucr("../FordA_TEST.txt")

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

min_val = X_train_tensor.min()
max_val = X_train_tensor.max()

X_train_tensor = (X_train_tensor - min_val) / (max_val - min_val)
X_test_tensor = (X_test_tensor - min_val) / (max_val - min_val)


train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

print(torch.min(X_train_tensor), torch.max(X_train_tensor))

Train shape: (3601, 500), Test shape: (1320, 500)
tensor(0.) tensor(1.)


In [5]:
n_splits = 20
split_size = len(test_data) // n_splits
flip_percs = np.arange(0.0, 0.51, 0.05)
accuracies1 = np.zeros((len(flip_percs), n_splits))
input_dim = torch.prod(torch.tensor(list(train_data[0][0].size())))
subset_size = 500
print(input_dim)
print(dim)

tensor(500)
15000


In [6]:
X_train_tensor.shape

torch.Size([3601, 500])

In [7]:
@torch.no_grad()                       # no autograd graph needed
def flip_rows_(tensor: torch.Tensor, perc: float, *, inplace: bool = True) -> torch.Tensor:
    """
    Flip the sign of `perc` fraction of elements **in every row** of `tensor`.

    Args
    ----
    tensor : torch.Tensor               # shape (N, D)
        2-D tensor whose rows will be sign-flipped.
    perc   : float                      # 0 ≤ perc ≤ 1
        Fraction of positions per row to flip.
    inplace: bool (default True)
        If True, modify `tensor` in place and return it;
        otherwise return a flipped clone.

    Returns
    -------
    torch.Tensor
        The tensor with sign-flipped rows (same object if `inplace` is True).
    """
    if not (0.0 <= perc <= 1.0):
        raise ValueError("`perc` must be in the interval [0, 1].")

    # Short-circuit trivial cases
    if perc == 0.0 or tensor.numel() == 0:
        return tensor
    if not inplace:
        tensor = tensor.clone()

    N, D = tensor.shape
    k = int(round(D * perc))            # exact count per row
    if k == 0:
        return tensor                   # nothing to flip

    device = tensor.device
    for r in range(N):
        # choose k unique positions in this row
        idx = torch.randperm(D, device=device)[:k]
        tensor[r, idx] *= -1            # in-place sign change

    return tensor


In [10]:
for i, perc in enumerate(flip_percs):
    print(f'Flipping Percentage: {perc} ------------')
    print('input_dim: ', input_dim)
    encoder = RandomFourierEncoder(input_dim, gamma, gorder, output_dim=dim)
    encoder.build_item_mem()
    train_hd, y_train = encoder.encode_data_extract_labels(train_data)
    train_dataset = HDDataset(train_hd.cpu(), y_train.cpu())
    trainloader = DataLoader(train_dataset, batch_size=10, shuffle=True, pin_memory=False, num_workers=0)
    for split_idx in range(n_splits):
        # Test
        print(f'Split Index: {split_idx+1}')
        split_indices = np.random.choice(len(test_data), size=subset_size, replace=False)
        split_subset = Subset(test_data, split_indices)
        test_hd, y_test = encoder.encode_data_extract_labels(split_subset)
        flip_rows_(test_hd, perc=perc) 
        test_dataset = HDDataset(test_hd, y_test)
        testloader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory=False, num_workers=0)
        accuracies1[i, split_idx] = train(trainloader, testloader, lr, dim)
        del testloader
        del test_dataset
        del test_hd
        
    del train_hd
    del trainloader
    del train_dataset
    print(f'Average Accuracy for flip perc={perc}: {np.mean(accuracies1[i, :])}')

Flipping Percentage: 0.0 ------------
input_dim:  tensor(500)
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split Index: 6
Split Index: 7
Split Index: 8
Split Index: 9
Split Index: 10
Split Index: 11
Split Index: 12
Split Index: 13
Split Index: 14
Split Index: 15
Split Index: 16
Split Index: 17
Split Index: 18
Split Index: 19
Split Index: 20
Average Accuracy for flip perc=0.0: 59.720000000000006
Flipping Percentage: 0.05 ------------
input_dim:  tensor(500)
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split Index: 6
Split Index: 7
Split Index: 8
Split Index: 9
Split Index: 10
Split Index: 11
Split Index: 12
Split Index: 13
Split Index: 14
Split Index: 15
Split Index: 16
Split Index: 17
Split Index: 18
Split Index: 19
Split Index: 20
Average Accuracy for flip perc=0.05: 54.60999999999999
Flipping Percentage: 0.1 ------------
input_dim:  tensor(500)
Split Index: 1
Split Index: 2
Split Index: 3
Split Index: 4
Split Index: 5
Split 

In [11]:
np.mean(accuracies1, axis=1)
# accuracies1

array([59.72, 54.61, 54.77, 56.51, 56.99, 54.91, 51.25, 52.22, 52.26,
       52.02, 40.46])

In [12]:
savemat('FordA_RFFHDC.mat', {'FordA_RFFHDC': accuracies1})